In [1]:
pip install xgboost==0.90

Note: you may need to restart the kernel to use updated packages.


In [2]:
from numpy import loadtxt
import pandas as pd
import matplotlib as plt
import requests,json
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
data_read = pd.read_csv('XBTUSD-1h-data.csv')
dataset = pd.DataFrame(data_read)

In [7]:
#data.tail()

In [53]:
Y = []
for i in range(0, len(dataset['timestamp'])-1): 
    if dataset['close'][i]>dataset['open'][i]:
        Y.append(1)
    else: Y.append(0)
Y.append(1)

In [54]:
t = 0
for i in range(len(Y)):
    if Y[i] != 0:
        t+=1
print(t)

29460


In [55]:
print(len(Y)-t)

28454


In [56]:
# split data into X and y
train_len = int(0.8*len(dataset['timestamp']))
X_train = dataset[:train_len]
Y_train = Y[:train_len]
X_test = dataset[train_len:]
Y_test = Y[train_len:]

In [57]:
print(len(dataset), len(Y))

57914 57914


In [58]:
print(len(X_test), len(Y_test))

11583 11583


In [59]:
X1_train = X_train.drop(labels=['timestamp', 'symbol', 'trades', 'volume', 'vwap', 'lastSize', 'turnover', 'homeNotional', 'foreignNotional'], axis=1)
X1_test = X_test.drop(labels=['timestamp', 'symbol', 'trades', 'volume', 'vwap', 'lastSize', 'turnover', 'homeNotional', 'foreignNotional'], axis=1)

In [60]:
for label in ['open', 'close', 'high', 'low']:
    X1_train[label] = pd.to_numeric(X1_train[label])
for label in ['open', 'close', 'high', 'low']:
    X1_test[label] = pd.to_numeric(X1_test[label])

In [61]:
normalized_X1_train=(X1_train-X1_train.min())/X1_train.std()
normalized_X1_test=(X1_test-X1_test.min())/X1_test.std()

In [62]:
# split data into train and test sets
#seed = 7
#test_size = 0.33
#X_train, X_test = train_test_split(normalized_X1, test_size=test_size, random_state=seed)
#y_train, y_test = train_test_split(normalized_Y1, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X1_train, Y_train)
# make predictions for test data
y_pred = model.predict(X1_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 51.46%


In [19]:
print(len(y_pred))

11583


In [34]:
t = 0
for i in range(len(Y_train)):
    if Y_train[i] != 0:
        t+=1
print(t)

2


In [35]:
print(max(y_pred))

0


In [23]:
print(len(Y_test))

11583


In [15]:
from sklearn.metrics import f1_score as f1

In [63]:
print(f1(y_pred, Y_test))

0.6706502636203866


In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
#Optimizer
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import make_scorer
#evaluation metrics
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

xgb_model = model.fit(X1_train, Y_train)
xgb_prediction = model.predict(X1_test)

print(confusion_matrix(Y_test, xgb_prediction))
print(classification_report(Y_test, xgb_prediction))

[[ 237 5510]
 [ 112 5724]]
              precision    recall  f1-score   support

           0       0.68      0.04      0.08      5747
           1       0.51      0.98      0.67      5836

    accuracy                           0.51     11583
   macro avg       0.59      0.51      0.37     11583
weighted avg       0.59      0.51      0.38     11583

